# SICSS Norrköping 2023: Text Analysis

_Hendrik Erz, Institute for Analytical Sociology | <hendrik.erz@liu.se> | Twitter: @sahiralsaid_

Welcome to the practical part of the Text Analaysis day! In this notebook, you will try out some of the methods covered in the morning lecture. In particular, the methods covered will be:

* tf-idf scores
* topic modeling
* Word2Vec

Below, you will see several exercises that cover most of the steps from an unprocessed text corpus to a final trained model, and, lastly the analysis step.

You will work on these examples in smaller groups with guidance from me.

## Preliminaries: Loading the Corpus

Your task in this exercise is to write a function that will return parts of the corpus in a way that the models we are using works with. Here, we will be working with a corpus of the **[States of the Union (SOTU)](https://en.wikipedia.org/wiki/State_of_the_Union) of the U.S. presidents**.

***

The first step is always to load the corpus. We will use a **generator** for this, since a generator helps us keep the memory footprint small and therefore to keep the model training times low.

Below you can find a ready-made function that will automatically return the corpus in the following format:

```python
corpus = [
    ('This is a speech from a republican', 'R'),
    ('This is a speech from a democrat', 'D'),
    # ...
]
```

As you can see, you will get from this generator function a list of **tuples**. The first element is always a speech, the second element is a letter indicating the president's party. The party codes are as follows:

* R: Republican
* D: Democrat
* W: Whig
* F: Federalist
* DR: Democratic-Republican
* na: No party
* NU: National Union

***

**Whenever you need the speeches, just call `speeches()` in your code**

In [99]:
def speeches ():
  """A generator that yields (speech, party) tuples"""
  # NOTE: Ensure that the file is in the same directory as this notebook.
  with open("sotu.csv", "r", encoding="utf-8") as fp:
    next(fp) # Skip the header
    for line in fp:
      speech, party = line.strip().split('\t')
      yield (speech, party)

In [ ]:
# Make sure we have all 251 speeches in our generator
sum([1 for x in speeches()])

## Preliminaries: Installing the necessary packages

This workbook relies on a set of packages. Depending on what package manager you use, there are various ways to install them (e.g., with Conda, pip, or virtualenv).

Make sure to install them according to how you have set up Python. If you use plain `pip`, here is how you can install them:

```bash
python -m pip install pandas
python -m pip install numpy
python -m pip install gensim
python -m pip install sklearn
python -m pip install nltk
python -m pip install matplotlib
```

## Getting to Know the Corpus

Before we dive into the corpus, we should make sure we have some bearings in the corpus. For this, a very simple method is to simply start counting words. This is not necessarily the most informative method, but it will give us some data that we can use to make some sense of the corpus.

Getting to some word counts involves a few steps:

1. First, we need a **tokenizer** that can split up a speech into tokens.
2. After we have a method of splitting a document into its constitutent words, we can use this information to build a vocabulary.
3. We can count words to ensure our corpus looks as it should.

### Exercise 1: Write a text tokenizer

Below, write a function that takes a single speech and returns a list of tokens. You should remove digits, punctuation marks, and other symbols that do not comprise regular, English words. Also, ensure that capitalization of letters does not matter by transforming the speech into lower case first.

> TIP: The NLTK package offers a lot of useful functions for working with natural language. It includes functions to remove stopwords and to tokenize a text. Also, the String class of Python provides additional easy functions you can use.

In [38]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Loads a common english stopword list
stops = stopwords.words('english')

def tokenize_speech(speech):
  raise NotImplementedError()

In [ ]:
# Run this cell to see how the preprocessor processes the first speech.
first_speech = next(speeches())[0]
tokenize_speech(first_speech)[0:10] # Look at the first ten words

### Exercise 2: Build a Vocabulary

We will need a vocabulary for many of the following steps, so it makes sense to write a dedicated function for it. The easiest form of a vocabulary is a dictionary that maps words to indices:

```python
vocab = {
    'word': 0,
    'word2': 1,
    # ...
}
```

Since we also need to figure out words by their indices after running some of our methods, we should create a so-called `i2w`-dictionary. The `i2w` ("index to word") performs the reverse lookup and maps indices to words:

```python
iw2 = {
    0: 'word',
    1: 'word2',
    # ...
}
```

We can then use these dictionaries to retrieve indices or words, depending on what we need:

```python
vocab['president']
# > May yield '24'

word = i2w[24]
# > May yield 'president'
```

Below, write a function that returns both a vocab and an i2w.

In [40]:
def build_vocab ():
  raise NotImplementedError()

In [46]:
# Run this cell to create the vocabulary
vocab, i2w = build_vocab()

In [ ]:
# Let us have a first look at the first ten words in our vocabulary. Note how
# the first ten words in our vocab equal the first words of the first speech.
list(vocab.keys())[0:10]

### Exercise 3: Count words

The final introductory exercise involves counting words. Python luckily has a useful function for us: `Counter`.

In [47]:
# Below, write a function that can count the words and returns a list of words
# and their frequencies.
from collections import Counter

def count_words ():
  raise NotImplementedError()

In [ ]:
# Let us now look at the ten most common, and then ten least common words:

words = count_words()

print("Most common words:")
print(words[0:10])

print("")
print("Least common words:")
print(words[-10:])

### Verify Zipf's Law

One great way of ensuring that your corpus does not include oddities and spurious data, a very simple verification involves checking for Zipf's law: The fact that a few words are very common, and many words are relatively uncommon. A histogram of words should resemple a logarithmic function.

Below, plot the frequencies ordered by rank.

In [ ]:
import matplotlib.pyplot as plt

# Extract only the actual numbers, and sort them by size descending.
word_counts = sorted(list(map(lambda x: x[1], words)), reverse=True)

plt.plot(range(len(word_counts)), word_counts)
plt.yscale("log") # Set the Y-axis to a logarithmic scale.
plt.ylabel("Frequency")
plt.xlabel("Rank")
plt.title("Zipf's Law: State of the Union Corpus")
plt.show()

## Computing tf-idf scores

The most simple way to begin an analysis is by calculating tf-idf scores. You can do this "manually", but for practical usage, there are some libraries that already do that for you.

Calculating tf-idf scores consists of three steps:

1. Calculate the term frequencies of words within each document
2. Calculate the inverse document frequency for each word
3. For each word in each document, calculate tf-idf by multiplying the document-specific frequency with the word's IDF-score

Remember, tf-idf is defined as:

$$
{\displaystyle \text{tf-idf} (t, d, D) = \mathrm{tf} (t,d) \times \mathrm{idf}}(t, D)
$$

where

$$
{\displaystyle \mathrm {tf} (t,d)={\frac {f_{t,d}}{\sum _{t'\in d}{f_{t',d}}}}}
$$

with $t$ = the term in question, $t'$ = all other terms, and $f_t$ = the (document)relative frequency of the term. And:

$$
 \mathrm{idf}(t, D) =  \log \frac{N}{1 + D}
$$

with $N$ = total number of documents in the corpus and $D$ = number of documents that contain term $t$.

### Exercise 4: Calculate tf-idf scores

Below, write a function that calculates tf-idf scores for our corpus and then outputs the most relevant word for each speech.

> The library `scikit-learn` has a useful function for that called `TfidfVectorizer`. You will need to pass it the vocabulary and the speeches.

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer

def calculate_tf_idf ():
  raise NotImplementedError()

In [ ]:
# Call the function and calculate the tf-idf scores
tf_idf = calculate_tf_idf()

### Exercise 5: Analysis of tf-idf scores

Now that we have calculated tf-idf scores, it is time to have a look at them. Below, write code to print out the highest scoring word for each speech, as well as, afterwards, the lowest-scoring word.

Explain what makes the words important or unimportant, and what this means in the context of the SOTU corpus.

In [ ]:
print("Most important words (according to tf-idf):")
# TODO: Your code here

print("")
print("Least important words (according to tf-idf):")
# TODO: Your code here

## Running a Topic Model

The next step to see what is inside our corpus is to run a topic model. The most common model is Latent Dirichlet Allocation (LDA). The library `sklearn` already provides such a model. However, again, here we have to preprocess the sentences. However, this time, we have to do it differently.

Running an LDA model requires a so-called Document-Term Matrix (DTM). In it, documents are defined as "one hot"-vectors. The matrix has the shape `(number of documents, number of words)`, and each cell is set to `0` if the document does not contain the word, and `1` if it does.

With the `tokenize_speech()` function from above, we already have a function that splits speeches into tokens. We now just need to build the DTM based on that. Building a DTM consists normally of these steps:

1. Create a vocabulary that contains every token within the whole corpus
2. Optionally, remove the most often occurring and the least often occurring terms to reduce the amount of words
3. Go over the corpus and set the corresponding cells in the matrix to `1`, if the document contains a word in the vocabulary.

### Exercise 6: Build a DTM

Below, write a function that creates a DTM. We have already provided a matrix that is set to all zeros and can be fed into the LDA function.

In [86]:
import numpy as np

def build_dtm ():
  raise NotImplementedError()

### Exercise 7: Run the Topic model

Below, write a function that trains a topic model. I have already added the correct function import for you. One thing that you will need to do, however, is figure out three hyperparameters: K, alpha, and beta.

Since we are dealing with a small corpus, let us just set $K = 1$. However, you still need to figure out a good alpha and a good beta. Beta should normally be larger than alpha, and both should be smaller than 0.5. Feel free to run the model several times while doing exercise 6 to figure out good values.

In [87]:
from sklearn.decomposition import LatentDirichletAllocation

def fit_lda_model ():
  raise NotImplementedError()

In [88]:
# Train a model
model = fit_lda_model()

### Exercise 8: Analyze the topic model

The last step in this exercise is to analyse the topic model. The most common method is to simply output the most important words (here, let us use 10 words) for each topic and see if you can make out any semantic topics.

> Below, write a function that prints the ten most important words for each topic. Try to think of fitting topic labels. What topic do the words seem to focus on? TIP: In order to sort the words correctly, you can use the functions `np.argsort` and, afterwards, `np.fliplr`, to reverse the order of the top words. Additionally, the topic-term-matrix is accessible with the property `components_` of the trained model. The shape of this matrix is `(n_topics, n_words)`.

In [89]:
def print_top_words (model):
  raise NotImplementedError()

In [ ]:
# Call the function
print_top_words(model)

## Running Word2Vec

The most advanced NLP method we will cover today is a Word2Vec model. Such a model encodes co-occurrence patterns of words in so-called word embeddings, vectors of numbers with 50, 100, 200, or 300 dimensions.

Here, you will write the least code since we will be using the gensim-library to run Word2Vec. Despite it being the (mathematically) most complex model, it can be run very fast and efficiently.

### Exercise 9: Run Word2Vec

Below, write code that uses gensim's Word2Vec model and run it on our corpus.

Train two models, one with a `window size` of 5, and one with 30.

In [102]:
from gensim.models import Word2Vec

# TODO: Your code here

### Exercise 10: Analyze the word embeddings

As a last exercise for today, here we analyze the word embeddings. Word embeddings are very good to find out what words are related to others. On gensim's model, you can check so by utilizing `model.wv.most_similar('word')`.

> Below, print out the most similar words for `america`, `government`, `bank`, and `war` for both models.

In [ ]:
# TODO: Your code here

## Conclusion

This concludes today's introduction into text analysis methods. Notice how we produced almost no plots: These methods are all great for getting a bearing on some corpus. Once you have found some interesting patterns, it is time to throw the results of one or more of the above methods into a model and test hypotheses with it.

Regardless of whether you have used the provided corpus, or your own: Try to come up with some research questions over the afternoon that you could check. Maybe this leads to a great project idea?

If you have further questions regarding these methods, do not hesitate to send me a mail to <hendrik.erz@liu.se> or ping me on Twitter <https://twitter.com/sahiralsaid>!